# Onderzoeksvraag 3: Kun je op basis van bepaalde keywords in de beschrijving een voorspelling doen over hoe hoog de score van deze wijn is? (Supervised - Regression/ Datavisualisatie)
We willen onderzoeken of je de score van de wijn kunt bepalen aan de hand van sommige keywords die gevonden zijn in de titels en de beschrijving.

# 1.0 Data Prep
Voor dat we kunnen beginnen moeten we onderdelen van de data aanpassen en rechtzetten.
## 1.1 Imports
Eerst beginnen we met een aantal setting en imports te doen, deze zijn allemaal netjes gegroepeerd.
### Let op!

Is dit de eerste keer dat dit bestand gedraaid wordt?<br>
Uncomment dan `nltk.download()` in de cel hier onder en download dan alleen stopwords.


In [ ]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
# nltk.download()
# Only download stopwords!
stop_words = stopwords.words('english')

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

## 1.2 Data cleaning
Hier laden en schonen we de wijn data set op.

Er staat een dictionary met alle foute waardes in alle colommen, eerst vervangen we die met nan.<br>
Daarna verandere we de types naar het correcte type.<br>
Dit doen we zodat we later in het programma nog kunnen beslissen wat we met de nan waardes doen.

In [ ]:
wine = pd.read_csv('redwine.csv', delimiter=';')
chemColNames = ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']

colErrorPairs = {
    'density'    : [' . '],
    'citric acid': [' - ',' -   '],
    'alcohol'    : ['100.333.333.333.333','11.066.666.666.666.600','956.666.666.666.667','923.333.333.333.333']}

for colName in colErrorPairs:
    for faultyString in colErrorPairs[colName]:
        wine[colName] = wine[colName].replace(faultyString,np.nan)
        
wine['alcohol'] = wine['alcohol'].astype(float)
wine['density'] = wine['density'].astype(float)
wine['citric acid'] = wine['citric acid'].astype(float)

## 1.3 Data wrangeling

Hier proberen we de informatie te halen uit de data die we willen en in de vorm die we willen.<br>
Het doel hier is alle text in `title` & `description` halen zodat we kunnen zoeken hoevaak sommige woorden genoemd worden.
<br><br>
Eerst selecteren we alleen `title` & `description`.<br>
We proberen een grote lijst met losse woorden te maken om unieke woorden te kunnen tellen.<br>
Per wijn splitten we de woorden naar losse strings, deze lijst maken we plat met `flatten` hier vervangen we alle delimiters.

In [ ]:
wordsPerWine = wine[['title','description']]
allWordsSeperated = np.array(' '.join(wordsPerWine.to_numpy().flatten()).replace(',','').split(" "))
allWordsSeperated = np.char.lower(allWordsSeperated)

Hier kan je zien dat alle woorden los zijn gehaald.<br>
Met een lengte van 118188 totale woorden.

In [ ]:
(allWordsSeperated[:20],'Amount of words',allWordsSeperated.shape)

Hier initialiseren we het `wordFrame`, dit is een functie geworden omdat we later dingen gaan laten vallen en weer snel een nieuw frame willen.<br>
<br>
Belangrijk, eerst tellen we alle unieke woorden, hier komen 2 lijsten uit.
`unique`: een lijst met alleen unieke woorden en `counts`: hoevaak dit woord getelt is.<br>
Daarna wordt het aan een nieuw frame toegevoegd en de colommen en types goed gezet.

In [ ]:
unique, counts = np.array(np.unique(allWordsSeperated, return_counts=True))
def initWordFrame():
    wordCount = np.array([np.array(unique), np.array(counts)])
    wordFrame = pd.DataFrame(wordCount[0],wordCount[1]).reset_index()
    wordFrame.columns = ['count','word']
    wordFrame['count'] = wordFrame['count'].astype(int)
    return wordFrame
wordFrame = initWordFrame()

Hier filteren we op woorden die minimaal `minCount` keer genoemd moeten zijn en ook niet in de stopwoorden lijst zit.

In [ ]:
print(f'Total of unique seperated words: {wordFrame.shape[0]}')
minCount = 400
wordFrame.drop(wordFrame[ (wordFrame['word'].isin(stop_words)) | (wordFrame['count'] < minCount)].index , inplace=True)
print(f'Total of unique seperated words with atleast a count of {minCount} & not a stopword: {wordFrame.shape[0]}')
# wordFrame.sort_values(by=['count'], ascending=False).reset_index(drop=True)[:32]

Dit is de verdeling van alle woorden die minimaal `400` keer genoemd zijn.<br>
Onze redenatie hierachter is dat er sommige woorden misschien een groep aangeven of een gedacht/smaak en dat deze vaken zouden voorkomen.<br>
Dit geeft ook een mooi max van `32` woorden aan.

In [ ]:
wordFrame.boxplot(column='count')

Hier maken we een nieuwe colom aan in de initiele `wine` tabel waar het het aantal gevonden keywords in een wijn kunnen aangeven per wijn.<br>
<br>
Het aantal keywords in de `title` en de `description` worden geteld en toegevoegd aan `keyword_count`.

In [ ]:
wine['keyword_desc'] = wine['description'].str.count('|'.join(wordFrame['word'].to_list()))
wine['keyword_title'] =  wine['title'].str.count('|'.join(wordFrame['word'].to_list()))
wine[['points', 'keyword_desc', 'keyword_title', 'title','description']]

Voordat we kunnen trainen met een lineair regressie model moeten we de data splitten dat doen we met een bestaande functie.

In [ ]:
features = ['keyword_desc', 'keyword_title']

# Feature waardes
X = wine[features]
X.fillna(X.mean(),inplace=True)

# Target waardes
y = wine['points']
y.fillna(y.mean(),inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=6)

# 2.0 Model fitting
Hier zullen we de gewonnen data toepassen op een lineair regressie model.
## 2.1 Model fitten met waarde uit hypothese.

In [ ]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)
score = linreg.score(X_test,y_test)
yPred = linreg.predict(X_test)
meanError = np.sqrt(mean_squared_error(y_test,yPred))
print(f'Score: {score}, Mean Error: {meanError}')

## 2.2 Model testen met aandere waardes
De lage score van het model hiervoor van `0.10xx` en de handmatig gekoze minimale woord waarde is aanleiding tot dit stuk code.<br>
Hier proberen we meerder malen een nieuwe model te maken en het te trainen maar dan met aandere mimimale keywoord count.<br>
We beginnen met een hoog minimum en we gaan geleidelijk minderen en zo voegen we woorde toe om het verschil te zien.

In [ ]:
maxRange = 22
minRange = 4
maxNumberRanges = [x*x for x in reversed(range(minRange,maxRange))]

# Target waardes
y = wine['points']
y.fillna(y.mean(),inplace=True)

for minCount in maxNumberRanges:
    
    wordFrame = initWordFrame()
    wordFrame.drop(wordFrame[ (wordFrame['word'].isin(stop_words)) | (wordFrame['count'] < minCount)].index , inplace=True)
#     Veranderd niks aan de score
#     wordFrame['count'] = (wordFrame['count'] - wordFrame['count'].mean())/wordFrame['count'].std(ddof=0)

    print(f'Total of unique seperated words with atleast a count of {minCount} & not a stopword: {wordFrame.shape[0]}')
    
    wine['keyword_desc'] = wine['description'].str.count('|'.join(wordFrame['word'].to_list()))
    wine['keyword_title'] =  wine['title'].str.count('|'.join(wordFrame['word'].to_list()))
    
    # Feature waardes
    X = wine[features]
    X.fillna(X.mean(),inplace=True)

    

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=6)
    
    linRegTrial = LinearRegression()
    linRegTrial.fit(X_train, y_train)
    score = linRegTrial.score(X_test,y_test)
    yPred = linRegTrial.predict(X_test)
    meanError = np.sqrt(mean_squared_error(y_test,yPred))
    print(f'Score: {score}, Mean Error: {meanError}\n')

# Review
Even een herhaling van de onderzoeksvraag.<br>
Kun je op basis van bepaalde keywords in de beschrijving een voorspelling doen over hoe hoog de score van deze wijn is?

## Verloop
Verloop van het onderzoek.

Mijn verwachting is dat als je een betere definitie hebt van 'niche' woorden, woorden die wat speciaals betekenen zoals 'grand cru' dat je een hogere score kan behalen.<br>
Wat er nog had kunnen gebeuren is kleinere ranges met woorden gebruiken.<br> 
Eerst wordt ge-fit met een kleine groep woorden `n=32` van `minimum=400`, n wordt hier steeds groter door de 'versoepeling' van het aantal hits van deze keywords.<br>
Dit hadden we ook vanaf de andere kant kunnen benaderen, zo hadden de veel gebruikte woorden weggelaten kunnen worden en hadden er gemakkelijker slices <b><i>tussen</i></b> de aantallen genomen kunnen worden.

## Conclusie

De conclusie die ik hier uit trek is dat het zeker mogelijk is maar nu nog niet goed genoeg, je ziet dat er een relatie is tussen <b><i>welke</i></b> woorden er worden geteld en de score.<br>
En dat met een beter systeem,(misschien punten waardes geven aan alle woorden) er een beter resultaat uit komt.